In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
imp_data = pd.read_csv("../data/longimp1/import.csv")
imp_data.head()

,Country,Time,Price Index
0,Canada,19-Dec,138.0
1,Canada,20-Jan,137.1
2,Canada,20-Feb,130.8
3,Canada,20-Mar,120.5
4,Canada,20-Apr,106.3


In [17]:
imp_data['Time'] = pd.to_datetime(imp_data['Time'], format="%y-%b")

In [18]:
policy_data = pd.read_csv("../data/OxCGRT_latest.csv")
policy_data.head()

/Users/wenqianzhang/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200101,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200102,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200103,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200104,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,ABW,NaN,NaN,NAT_TOTAL,20200105,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
policy_data.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'H8_Protection of elderly people', 'H8_Flag', 'M1_Wildcard',
       'V1_Vaccine Prioritisation (summary)',
       'V2A_Vaccine Availability (summary)',
       'V2B_Vaccin

In [82]:
cols = ['CountryName', 'RegionName', 'Date', 'C1_School closing', 'C2_Workplace closing', 'C3_Cancel public events', 
        'C4_Restrictions on gatherings', 'C5_Close public transport', 'C6_Stay at home requirements',
       'C7_Restrictions on internal movement', 'C8_International travel controls', 
        'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex']
policy_data_new = policy_data[cols].copy()

In [83]:
countries = ['Canada', 'United States', 'Mexico', 'Japan', 'China']
policy_data_new = policy_data_new[policy_data_new['CountryName'].isin(countries)].copy()

In [84]:
policy_data_new['Date'] = pd.to_datetime(policy_data_new['Date'], format="%Y%m%d")

In [85]:
policy_data_new = policy_data_new.fillna(0)
policy_data_new['month'] = policy_data_new['Date'].apply(lambda x: str(x.year) + '-' + str(x.month))

In [86]:
agg_data = policy_data_new.groupby(['CountryName', 'month']).sum().reset_index()

In [87]:
poli = ['C1_School closing', 'C2_Workplace closing', 'C3_Cancel public events', 
        'C4_Restrictions on gatherings', 'C5_Close public transport', 'C6_Stay at home requirements',
       'C7_Restrictions on internal movement', 'C8_International travel controls']

for pol in poli:
    agg_data[pol] = agg_data[pol].apply(lambda x: 1 if x > 0 else 0)

In [88]:
agg_data[agg_data['month']=='2021-10']

,CountryName,month,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,ConfirmedCases,ConfirmedDeaths,StringencyIndex
13,Canada,2021-10,1,1,1,1,1,1,1,1,1.041114e+08,1759027.0,25382.57
41,China,2021-10,1,1,1,1,1,1,1,1,6.345485e+06,287432.0,50485.49
69,Japan,2021-10,1,1,1,1,0,1,1,1,5.321334e+07,559205.0,1463.82
97,Mexico,2021-10,1,1,1,0,1,1,1,1,1.161256e+08,8795166.0,1205.59
125,United States,2021-10,1,1,1,1,1,1,1,1,2.779251e+09,44658385.0,51515.39


In [89]:
imp_data['month'] = imp_data['Time'].apply(lambda x: str(x)[: 7])

In [97]:
data_all = pd.merge(imp_data, agg_data, left_on=['Country', 'month'], right_on=['CountryName', 'month'], how='left')
data_all.fillna(0, inplace=True)
data_all.drop(columns=['Time', 'CountryName'], inplace=True)

In [101]:
agg_data

,CountryName,month,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,ConfirmedCases,ConfirmedDeaths,StringencyIndex
0,Canada,2020-1,1,0,1,0,0,0,0,1,3.400000e+01,0.0,3460.96
1,Canada,2020-10,1,1,1,1,1,1,1,1,1.220498e+07,609200.0,25276.69
2,Canada,2020-11,1,1,1,1,1,1,1,1,1.823737e+07,674683.0,26010.10
3,Canada,2020-12,1,1,1,1,1,1,1,1,3.008762e+07,879720.0,28829.00
4,Canada,2020-2,1,0,1,0,0,0,0,1,4.220000e+02,0.0,4773.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,United States,2021-9,1,1,1,1,1,1,1,1,2.494556e+09,39871067.0,50617.44
136,United States,2022-1,1,1,1,1,1,1,1,1,4.073101e+09,52802767.0,55409.66
137,United States,2022-2,1,1,1,1,1,1,1,1,4.339352e+09,51501354.0,49058.19
138,United States,2022-3,1,1,1,1,1,1,1,1,4.919809e+09,59838975.0,48333.64


In [99]:
agg_data[agg_data['CountryName']=='China']

,CountryName,month,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,ConfirmedCases,ConfirmedDeaths,StringencyIndex
28,China,2020-1,1,1,1,1,1,1,1,1,75794.0,1778.0,19649.21
29,China,2020-10,1,1,1,1,1,1,1,1,5477123.0,287308.0,43903.98
30,China,2020-11,1,1,1,1,1,1,1,1,5365960.0,278040.0,45759.94
31,China,2020-12,1,1,1,1,1,1,1,1,5581133.0,287308.0,48858.20
32,China,2020-2,1,1,1,1,1,1,1,1,3263070.0,92760.0,71325.48
33,China,2020-3,1,1,1,1,1,1,1,1,5015237.0,196766.0,69672.82
34,China,2020-4,1,1,1,1,1,1,1,1,5008928.0,236340.0,61910.07
35,China,2020-5,1,1,1,1,1,1,1,1,5237624.0,287278.0,58794.73
36,China,2020-6,1,1,1,1,1,1,1,1,5098089.0,278040.0,53385.68
37,China,2020-7,1,1,1,1,1,1,1,1,5304449.0,287308.0,53555.49


In [98]:
data_all[data_all['Country'] == 'China']

,Country,Price Index,month,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,ConfirmedCases,ConfirmedDeaths,StringencyIndex
56,China,98.2,2019-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
57,China,98.2,2020-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
58,China,98.0,2020-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
59,China,98.0,2020-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
60,China,98.1,2020-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
61,China,98.0,2020-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
62,China,98.0,2020-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
63,China,98.2,2020-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
64,China,98.3,2020-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
65,China,98.3,2020-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
